In [9]:
from rex import WindX 
import pandas as pd
import numpy as np

### Notebook for processing WRF output data to obtain horizontal temperature gradients
- Requires additional package `nrel-rex`; see https://nrel.github.io/rex/misc/installation.html#option-1-install-from-pip-or-conda-recommended-for-analysts
- WRF data is downloadable from https://data.openei.org/s3_viewer?bucket=nrel-pds-wtk&prefix=Mid_Atlantic%2F, filename is `Mid_Atlantic_2020.h5`
- !! Note that this data file is nearly 200GB in size! It is recommended to process on an HPC platform. NREL users may find this data accessible on Eagle.

In [10]:
alts = [10,20,40,60,80,100,120,140,160,180,200,220,240,260]
lat_grid = [39.35, 39.55, 39.75]
lon_grid = [-73.63, -73.43, -73.23]
buoys = []
for i in range(len(lat_grid)):
    for j in range(len(lon_grid)):
        buoys.append((lat_grid[i], lon_grid[j]))


In [12]:
df = None
wtk_file = f'/datasets/WIND/Mid_Atlantic/Mid_Atlantic_2020.h5' # path on Eagle
with WindX(wtk_file) as f:
    for alt in alts:
        for buoy in buoys:
            ws_ = f.get_lat_lon_df(f'windspeed_{alt}m', buoy)
            ws_ = ws_.rename(columns={ws_.keys()[0]:'wind_speed'})
            
            wd_ = f.get_lat_lon_df(f'winddirection_{alt}m', buoy)
            wd_ = wd_.rename(columns={wd_.keys()[0]:'wind_direction'})
            ws_ = ws_.join(wd_)
            
            tm_ = f.get_lat_lon_df(f'temperature_{alt}m', buoy)
            tm_ = tm_.rename(columns={tm_.keys()[0]:'temperature'})
            ws_ = ws_.join(tm_)
            
            ws_['altitude'] = alt
            ws_['lat'] = buoy[0]
            ws_['lon'] = buoy[1]
            ws_ = ws_.set_index('altitude', append = True)
            ws_ = ws_.set_index('lat', append = True)
            ws_ = ws_.set_index('lon', append = True)
            
            df = pd.concat([df,ws_])
        print(str(alt))

10
20
40
60
80
100
120
140
160
180
200
220
240
260


In [17]:
df

,,,,wind_speed,wind_direction,temperature
time_index,altitude,lat,lon,,,
2020-01-01 00:00:00+00:00,10,39.35,-73.63,8.40,237.369995,8.86
2020-01-01 01:00:00+00:00,10,39.35,-73.63,8.45,235.660004,9.18
2020-01-01 02:00:00+00:00,10,39.35,-73.63,8.96,235.500000,9.65
2020-01-01 03:00:00+00:00,10,39.35,-73.63,10.29,245.440002,9.82
2020-01-01 04:00:00+00:00,10,39.35,-73.63,10.59,293.989990,8.58
...,...,...,...,...,...,...
2020-12-31 19:00:00+00:00,260,39.75,-73.23,6.68,327.720001,4.70
2020-12-31 20:00:00+00:00,260,39.75,-73.23,8.15,330.250000,4.59
2020-12-31 21:00:00+00:00,260,39.75,-73.23,8.28,340.540009,4.95


In [18]:
df.to_pickle('other_data/ws_wd_temp_2020.pkl')